In [3]:
import tensorflow as tf
import pandas as pd

In [4]:
df =pd.read_csv('sample11_2_1.csv') 

In [5]:
# データを学習用と評価用に分離
train = df[:1000]
test = df[1000:]

In [6]:
#入力データの定義
x=tf.placeholder(tf.float32,[None,2])
#プレースフォルダを用いて入力を定義する。x1,x2の2変数なので入力は２となる
#またデータの個数は任意の数が指定できるようにNoneを指定する

In [9]:
#重みをバイアスの定義
W = tf.Variable(tf.zeros([2,2],tf.float32))
b=tf.Variable(tf.zeros([1,2],tf.float32))
# Variableは、その名の通り変数。重み、バイアスは学習が進むに連れて徐々に値を変えるので変数として定義する
# 　初期値は、0としてもみは２×２行列、バイアスは１×２行列とする

In [10]:
# 入力データは１度値が設定されれば値が変更される事はない
#Tensorfolowではこのように計算グラフを定義した時は値が決まっていないが計算を実行する際には値が設定されて
# その後変更されないと言うものは「プレースフォルダー」、計算実行中に随時値が変更されるものを変数と言う

In [11]:
#出力の定義
#(1)入力に重みを掛け、バイアスを足す
#(2)ソフトマックス関数を通し確率に変換
#yはtype１の確率、type2の確率の配列
# matmuは行列の配列とみなして行列積を計算


In [19]:
y = tf.nn.softmax(tf.matmul(x,W)+b)
# tf.nn　ニューラルネットワークのsoftmax関数
# matmul行列の配列だとみなして行列の積を計算

入力に対して重みを掛け、バイアスを足し算している。
次にそれを活性化関数であるソフトマクス関数に渡しています。
ソフトマックス関数は、TensorFlowのライブラリで定義されている為、呼び出すだけで計算グラフが作成されます。
出力yは与えられた入力がtype1（赤丸）でありそうか、type2（青丸）でありそうかを確率として表現する　例えば[0.83,0.17]のようになる

### 学習処理の定義
ここまででニューラルネットワークはできた。次に重みとバイアスを調整する為の学習処理を定義する

In [27]:
# 正解データ
t = tf.placeholder(tf.float32,[None,2])
# プレースフォルダーを使用に正解データ、100%赤丸（0%青星）、１００％青星（0%赤丸）と言うデータを読み込めるようにする

In [28]:
# 交差エントロピー誤差を定義
# 各データに対して交差エントロピー値を計算し
# 次に全交差エントロピーを合計
# reduce_sumでは多次元の配列を一気に足すという事もできるが今回はわかりやすさの為２段階に分けた
cross_entropy = tf.reduce_sum(tf.reduce_sum(t*tf.log(y),1))

In [31]:
# 最適化処理を定義
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

# 定義した交差エントロピー誤差を最小化する最適化計算を定義する
#　最適化処理では、非常に複雑な事が行われているがTensorflowではライブラリとして用意されているので呼び出すだけで良い。
# 今回は、Adamと呼ばれる勾配法と呼ばれる改良手法を利用する

### 正答率計算

In [41]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(t,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

# tf.equal(x, y) →x==yの真理値を返す　bool型（True, False）
# tf.argmax(y,1)は、計算データで、tf.argmax(t,1)は、教師データ
# tf.reduce_meanは、平均値計算　
# tf.castは、trueは１.０　False0.0に変換する

argmaxについて
渡された配列のうち、最大要素のインデックスを返す。例えば[0.8,0.2]であれば0、[0.3,0.7]であれば1を返すとした場合、
これを出力y（計算されたもの）、教師データt（予めわかっているもの）について計算する。なお、y,tは実際には２次元配列なので軸１（列）単位で比較します。
結果は１次元配列になる

ergalについて
第１引数、第２引数が等しいかどうかをTrue、Falseで返す。結果の次元数は変わらずに今回は[True,False,True]という値になる

castについて
第１引数と第２引数のデータ型に変換する。Trueであれば1.0、Falseであれば0.0になる
ここまでくればreduce_meanを利用し平均値を計算できる。例えば[1.0,0.0,1.0]であれば0.666...という値が計算される


### 学習の実行

ここまででニューラルネットワーク、学習処理、また、正答率の計算が定義できた。
ここからは実際にデータを与え、評価を行っていく

In [44]:
# セッションを作成し変数を初期化する
#グラフの事項単位をセッション（Session）と言う
# 変数(Variable)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# まずセッションを作成する
# またセッションで利用する変数は初期化が必要
# これには　global_variables_initializer　という処理を実行する

In [45]:
# ニューラルネットワークを学習させる

In [53]:
for i in range(100):
# 訓練データの一部を取り出して実行する
    start = (i * 100)% 1000
    batch = train[start:].head(100)
    xs = batch[['x1','x2']]
    ts = batch[['t1','t2']]
    sess.run(train_step,{x: xs,t:ts})

ニューラルネットワークの学習は一度だけでなく繰り返し学習を行う。
その際に全部の訓練データを与えるのでなく一部を与える。このようにする主な理由は計算時間を短縮する為です。これをミニバッチという。
データの取り出しかたについてはpandasに関する事項ですが、少し補足するとstartは0,100,200のように100刻みの数値になる。
1000,1100になると「％1000」として余りの0,100と計算される。次にスライスを使用し「１００移行」「２００以降」のように訓練データの一部を取り出す。
最後必要なのは取り出したデータのうち１００個だけなので、headメソッドを使用し先頭１００個を取り出す。このようにして１００個のデータを取り出したあと、
入力と正解データに分離を行う

 ### ニューラルネットワークの評価

In [51]:
# 評価データを　用いて正答率を計算
print(sess.run(accuracy,{x:test[['x1','x2']],t:test[['t1','t2']]}))

0.46


学習したニューラルネットワークに対して未知のデータを与えた場合、正解率46％という成績になった